<a href="https://colab.research.google.com/github/sunlight2018/hands_on_ml3_notebooks/blob/main/notebooks/03_exercise_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Write a function that can shift an MNIST image in any direction (left, right, up, or down) by one pixel.⁠7 Then, for each image in the training set, create four shifted copies (one per direction) and add them to the training set. Finally, train your best model on this expanded training set and measure its accuracy on the test set. You should observe that your model performs even better now! This technique of artificially growing the training set is called data augmentation or training set expansion.

In [26]:
# ✅ Step 1: 导入库
from tensorflow.keras.datasets import mnist
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np

In [27]:
# ✅ Step 2: 加载 MNIST 数据
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 正规化 & 增加通道维度（灰度图 → 1 通道）
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

x_train = np.expand_dims(x_train, -1)  # (60000, 28, 28, 1)
x_test = np.expand_dims(x_test, -1)

In [28]:
# ✅ Step 3: 设置数据增强器
data_aug = ImageDataGenerator(
    rotation_range=1,
    width_shift_range=0.05,
    height_shift_range=0.05,
    zoom_range=0.05
)

In [29]:
# ✅ Step 4: 生成增强图像
augment_size = 10000

augment_generator = data_aug.flow(
    x_train, y_train,
    batch_size=augment_size,
    shuffle=True
)

x_augmented, y_augmented = next(augment_generator)

# 合并原始和增强数据
x_train_all = np.concatenate((x_train, x_augmented))
y_train_all = np.concatenate((y_train, y_augmented))

In [30]:
# ✅ Step 5: 扁平化数据，KNN 要用向量
x_train_all_flat = x_train_all.reshape((x_train_all.shape[0], -1))  # (70000, 784)
x_test_flat = x_test.reshape((x_test.shape[0], -1))                # (10000, 784)

In [31]:
# ✅ Step 6: 使用 KNN 训练模型
knn_clf = KNeighborsClassifier(n_neighbors=4, weights='distance')
knn_clf.fit(x_train_all_flat, y_train_all)

y_pred = knn_clf.predict(x_test_flat)
acc = accuracy_score(y_test, y_pred)

print(f"KNN accuracy with data augmentation: {acc:.4f}")

KNN accuracy with data augmentation: 0.9706


另一种方法，教材里的

In [37]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', as_frame=False)

X, y = mnist.data, mnist.target
x_train, x_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

In [38]:
from scipy.ndimage import shift

def shift_image(image, dx, dy):
  image = image.reshape((28, 28))
  shifted_image = shift(image, [dy, dx], cval= 0, mode= 'constant')
  return shifted_image.reshape([-1])


In [39]:
x_train_augmented = [image for image in x_train]
y_train_augmented = [label for label in y_train]

for dx, dy in ((-1, 0), (1, 0), (0, -1), (0,1)):
  for image, label in zip(x_train, y_train):
    x_train_augmented.append(shift_image(image, dx, dy))
    y_train_augmented.append(label)

x_train_augmented_np = np.array(x_train_augmented)
y_train_augmented_np = np.array(y_train_augmented)

In [40]:
shuffle_idx = np.random.permutation(len(x_train_augmented_np))
x_train_augmented_np_shuffle = x_train_augmented_np[shuffle_idx]
y_train_augmented_np_shuffle = y_train_augmented_np[shuffle_idx]

In [41]:
knn_clf = KNeighborsClassifier(n_neighbors= 4, weights= 'distance' )
final_model_augmentd= knn_clf.fit(x_train_augmented_np_shuffle, y_train_augmented_np_shuffle)

In [42]:
final_model_augmentd.score(x_test, y_test)

0.9763